In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1659546321938_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

#### Initialising the Spark session

In [3]:
# Data stored in Public S3 Bucket 'chicago-crime-mlc' as 'Chicago_Crimes_2012_to_2017.csv'
df = spark.read.csv('s3a://advertising-dataset-ml/Advertising.csv', header = True, inferSchema = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Number of rows and columns in the dataset
print((df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(200, 5)

In [5]:
# Printing the first two row
df.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(_c0='1', TV='230.1', Radio='37.8', Newspaper='69.2', Sales='22.1'), Row(_c0='2', TV='44.5', Radio='39.3', Newspaper='45.1', Sales='10.4')]

In [6]:
# Schema of the dataset
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: string (nullable = true)
 |-- TV: string (nullable = true)
 |-- Radio: string (nullable = true)
 |-- Newspaper: string (nullable = true)
 |-- Sales: string (nullable = true)

In [7]:
# Dropping the variables that will not contribute to the final model
df = df.drop("_c0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

#### Check the number of null values for each variable

In [10]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+---------+-----+
| TV|Radio|Newspaper|Sales|
+---+-----+---------+-----+
|  0|    0|        0|    0|
+---+-----+---------+-----+

#### Change the required columns from string type to numerical before passsing into the vector assembler

In [11]:
# Change the required columns from string type to numerical before applying the Vector Assembler 

df = df.withColumn("TV", df["TV"].cast(FloatType()))
df = df.withColumn("Radio", df["Radio"].cast(FloatType()))
df = df.withColumn("Newspaper", df["Newspaper"].cast(FloatType()))
df = df.withColumn("Sales", df["Sales"].cast(FloatType()))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
inputCols = [
    'TV',
    'Radio',
    'Newspaper'
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Initialise the vector assembler as 'assembler'

In [13]:
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
assembler = VectorAssembler(inputCols=inputCols, outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.ml.regression import RandomForestRegressor

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Initialise the RandomForestRegressor as 'rf'

In [16]:
rf = RandomForestRegressor(featuresCol = 'features', labelCol = 'Sales', seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Splitting the dataset into train and test

In [17]:
# Splitting the dataFrame into training and testing set
train, test = df.randomSplit([0.7, 0.3], seed = 2020)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Create the pipleing and input the stages

In [18]:
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
pipeline = Pipeline(stages=[assembler, rf])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Train the model
model = pipeline.fit(df)

# Make predictions
predictions = model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Select example rows to display.
predictions.select("Sales").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|Sales|
+-----+
|  3.2|
|  5.5|
|  7.2|
|  7.3|
|  5.3|
+-----+
only showing top 5 rows

In [22]:
predictions.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['TV', 'Radio', 'Newspaper', 'Sales', 'features', 'prediction']

#### Using the model created, calculate the feature importances

In [23]:
# Defining a function to extract features along with the feature importance score
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Printing the feature importance scores
ExtractFeatureImp(model.stages[-1].featureImportances, predictions, "features").head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   idx       name     score
0    0         TV  0.496370
1    1      Radio  0.373478
2    2  Newspaper  0.130152

In [25]:
from pyspark.mllib.evaluation import RegressionMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Use RegressionMetrics to evaluate this Random Forest model

In [26]:
#important: need to cast to float type, and order by prediction, else it won't work:
preds_and_labels = predictions.select(['prediction','Sales']).withColumn('label', F.col('Sales').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = RegressionMetrics(preds_and_labels.rdd.map(tuple))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Squared Error
print("MSE = %s" % metrics.meanSquaredError)
print("RMSE = %s" % metrics.rootMeanSquaredError)

# R-squared
print("R-squared = %s" % metrics.r2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MSE = 1.8263428069925502
RMSE = 1.3514225123892787
R-squared = 0.9381381716050834